In [ ]:
# !pip install -e ".[dev]"

In [ ]:
# %load_ext autoreload
# %autoreload 2
# %env ANYWIDGET_HMR=1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dimbridge import Dimbridge

plt.style.use("ggplot")
plt.style.use("seaborn-v0_8-colorblind")

In [ ]:
x = np.random.rand(5000) * 5
y = np.random.rand(5000) * 2
z = np.cos(x + y)
w = x**2

df = pd.DataFrame(dict(d1=x, d2=y, d3=z, d4=w))

In [ ]:
# from umap import UMAP
# xy = UMAP(n_neighbors=50).fit_transform(np.c_[x, y, z, w])
xy = np.c_[x, y]

In [ ]:
# Testing
from importlib import reload

import dimbridge

reload(dimbridge)
from dimbridge import Dimbridge

dimbridge = Dimbridge(
    data=df,
    x=xy[:, 0],
    y=xy[:, 1],
    # c=np.repeat([[255,0,0]], xy.shape[0], 0).astype(np.int32),
    # s=np.zeros_like(x) + 10,  # mark size
    predicate_mode="data extent",
    brush_mode="curve", #"single", "contrastive" or "curve"
    # predicate_mode='predicate regression',
)
dimbridge

In [ ]:
print(dimbridge.selected)

In [ ]:
plt.scatter(df["d2"][dimbridge.selected], df["d3"][dimbridge.selected], s=10)

In [ ]:
# todo export svg / pdf
# todo how-to, readme
# video on website